In [ ]:
import pandas as pd
import database


In [ ]:
conn = database.db_connection()
cursor = conn.cursor()

In [ ]:
df = pd.read_csv("initial_queries.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df["client_mobile"] = df["client_mobile"].astype("object")

In [ ]:
df["query_created_time"] = df["query_created_time"].replace(["","\\N"],None)
df["query_closed_time"] = df["query_closed_time"].replace(["","\\N"],None)

In [ ]:
df["query_created_time"] = pd.to_datetime(df["query_created_time"], errors="coerce")
df["query_closed_time"] = pd.to_datetime(df["query_closed_time"], errors="coerce")

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.to_csv("initial_queries.csv",index=False)

In [99]:
df[df["status"] == "Open"].head()

,query_id,client_email,client_mobile,query_heading,query_description,status,query_created_time,query_closed_time
39,Q0040,hyoung@example.com,2052334703,Payment Failure,Charged twice for one payment.,Open,2025-07-20,NaT
168,Q0169,nancy39@example.net,9744030885,Account Suspension,Terms of use were unclear.,Open,2025-03-15,NaT
324,Q0325,robert76@example.net,8016153926,UI Feedback,Font inconsistent across pages.,Open,2025-03-30,NaT
363,Q0364,veronicameza@example.com,9222184573,Technical Support,Can’t sync data across devices.,Open,2025-03-08,NaT
373,Q0374,kenneth08@example.org,3573995383,Feature Request,Multi-user role system would help.,Open,2025-05-19,NaT


In [ ]:
conn.rollback()
for _, r in df.iterrows():
    cursor.execute("""
        INSERT INTO queries (query_id, client_email, client_mobile, query_heading,
                             query_description, status, query_created_time, query_closed_time)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        r["query_id"],
        r["client_email"],
        r["client_mobile"],
        r["query_heading"],
        r["query_description"],
        r["status"],
        r["query_created_time"],
        None if pd.isna(r["query_closed_time"]) else r["query_closed_time"]
    ))
conn.commit()